In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os
import zipfile

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

100%|██████████| 2.70G/2.70G [00:24<00:00, 116MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2


In [ ]:
# Kaggle Dataset Download
os.environ['KAGGLE_CONFIG_DIR'] = os.path.expanduser('~/.kaggle')

# Dataset identifier from Kaggle
dataset = "vipoooool/new-plant-diseases-dataset"
os.system(f"kaggle datasets download -d {dataset}")

# Unzip the dataset
with zipfile.ZipFile(f"{dataset.split('/')[-1]}.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")

print("Dataset downloaded and extracted successfully!")

Dataset downloaded and extracted successfully!


In [ ]:
# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
# Load the VGG-16 model with pre-trained weights, excluding the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Freeze the base model layers to retain pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:

# Create the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(38, activation='softmax')  # Adjust the number of classes as per your dataset
])

In [ ]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split for training and validation

)

In [ ]:
# Download the dataset using Kaggle API
!kaggle datasets download -d {dataset}

# Unzip the dataset
!unzip -q {dataset.split('/')[-1]}.zip -d dataset



Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
new-plant-diseases-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
replace dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Apple___Apple_scab/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Apple___Apple_scab/01a66316-0e98-4d3b-a56f-d78752cd043f___FREC_Scab 3003.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Apple___Apple_scab/01a66316-0e98-4d3b-a56f-d78752cd043f___FREC_Scab 3003_270deg.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Apple___Apple_scab/01

In [ ]:
#Verify if the directory exists
import os
if os.path.exists('dataset/train'):
  print("Directory 'dataset/train' exists.")
else:
  print("Directory 'dataset/train' does not exist.")

# Continue with the ImageDataGenerator flow_from_directory
train_generator = train_datagen.flow_from_directory(
    'dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train',  # Updated path
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train', # Updated path
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Directory 'dataset/train' does not exist.
Found 56251 images belonging to 38 classes.
Found 14044 images belonging to 38 classes.


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1757/1757 ━━━━━━━━━━━━━━━━━━━━ 964s 537ms/step - accuracy: 0.0484 - loss: 3.5397 - val_accuracy: 0.1154 - val_loss: 3.0662
Epoch 2/10
   1/1757 ━━━━━━━━━━━━━━━━━━━━ 3:41 126ms/step - accuracy: 0.0625 - loss: 3.4416

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1757/1757 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.0625 - loss: 3.4416 - val_accuracy: 0.1071 - val_loss: 2.8831
Epoch 3/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 908s 512ms/step - accuracy: 0.0644 - loss: 3.3172 - val_accuracy: 0.1221 - val_loss: 2.9989
Epoch 4/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 0s 141us/step - accuracy: 0.0625 - loss: 3.2406 - val_accuracy: 0.1429 - val_loss: 3.0006
Epoch 5/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 928s 517ms/step - accuracy: 0.0711 - loss: 3.2814 - val_accuracy: 0.1080 - val_loss: 2.9987
Epoch 6/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 0s 147us/step - accuracy: 0.0312 - loss: 3.2570 - val_accuracy: 0.1071 - val_loss: 3.0401
Epoch 7/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 901s 511ms/step - accuracy: 0.0778 - loss: 3.2530 - val_accuracy: 0.0911 - val_loss: 3.0621
Epoch 8/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 1s 233us/step - accuracy: 0.0312 - loss: 3.2799 - val_accuracy: 0.1429 - val_loss: 3.0541
Epoch 9/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 902s 512ms/step - accuracy: 0.0841 - los

In [ ]:
# Save the model
model.save('crop_disease_model.h5')



In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

439/439 ━━━━━━━━━━━━━━━━━━━━ 183s 414ms/step - accuracy: 0.1006 - loss: 3.0722
Validation Accuracy: 9.32%
